# Movie Rating Hypothesis testing

#### Grading:


- Code: 90 pts
- Markdown Documentation: 10 pts


In this assignment we are going prove of disprove a hypothesis

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from IPython.display import HTML, display

pd.set_option('display.notebook_repr_html', False)
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 25)

In [2]:
%matplotlib inline

##### Read and cleanup the movie data

In [3]:
def get_movie_data():
    
    unames = ['user_id','gender','age','occupation','zip']
    users = pd.read_table(os.path.join('../data','users.dat'), 
                          sep='::', header=None, names=unames, encoding='latin-1')
    
    rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
    ratings = pd.read_table(os.path.join('../data', 'ratings.dat'), 
                            sep='::', header=None, names=rnames, encoding='latin-1')
    
    mnames = ['movie_id', 'title','genres']
    movies = pd.read_table(os.path.join('../data', 'movies.dat'), 
                           sep='::', header=None, names=mnames, encoding='latin-1')

    return users, ratings, movies

users, ratings, movies = get_movie_data()

tmp = movies.title.str.extract('(.*) \(([0-9]+)\)')
tmp.apply(lambda x:x[0] if len(x) > 0 else None)
tmp.apply(lambda x: x[0][:40] if len(x) > 0 else None)

movies['year'] = tmp[1]
movies['short_title'] = tmp[0]

print(movies.head())

C:\Users\varsh\AppData\Local\Temp\ipykernel_24924\2599110875.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  users = pd.read_table(os.path.join('../data','users.dat'),
C:\Users\varsh\AppData\Local\Temp\ipykernel_24924\2599110875.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings = pd.read_table(os.path.join('../data', 'ratings.dat'),


   movie_id                               title                        genres  \
0         1                    Toy Story (1995)   Animation|Children's|Comedy   
1         2                      Jumanji (1995)  Adventure|Children's|Fantasy   
2         3             Grumpier Old Men (1995)                Comedy|Romance   
3         4            Waiting to Exhale (1995)                  Comedy|Drama   
4         5  Father of the Bride Part II (1995)                        Comedy   

   year                  short_title  
0  1995                    Toy Story  
1  1995                      Jumanji  
2  1995             Grumpier Old Men  
3  1995            Waiting to Exhale  
4  1995  Father of the Bride Part II  


C:\Users\varsh\AppData\Local\Temp\ipykernel_24924\2599110875.py:12: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies = pd.read_table(os.path.join('../data', 'movies.dat'),


### 1. Prove or disprove the following Null Hypothesis:

#### H0: Different age segments of users rate the different genres of movies similarly

- Age segments are (0-20), (21-30), (31-50), (50 and above),
- Genres - Drama, Comedy, Action, Romance, Adventure
- Rating 4 or above is a positive, else negative (used for binary votes)

#### Solution

#### Im joining the movies and ratings dataframes and setting the index a movie_id to create dummies to seperate the genres column
> ***movies_ratings*** dataframe

In [4]:
movies_ratings = (ratings
                  .set_index("movie_id")
                  .join(movies.set_index("movie_id"),
                        how="left")
                 )
movies_ratings.shape

(1000209, 7)

#### Creating seperate columns for genres by using get_dummies as they are all in one column seperated by pipe delimiter
> ***genres_dummies*** dataframe

In [5]:
genres_dummies = movies_ratings['genres'].str.get_dummies()
genres_dummies.shape

(1000209, 18)

#### Concatenate these dummies to the original movies_ratings data frame
> **full_movies_ratings** dataframe
- Also dropping genres other than Drama, Comedy, Action, Romance, Adventure

In [6]:
full_movies_ratings = (pd.concat([movies_ratings, genres_dummies], axis=1)
                       .drop(["timestamp","Animation","Children's","Crime","Documentary","Fantasy","Film-Noir","Horror","Musical","Mystery","Sci-Fi","Thriller","War","Western"], axis=1)
                )

full_movies_ratings.head()

          user_id  rating             title                       genres  \
movie_id                                                                   
1               1       5  Toy Story (1995)  Animation|Children's|Comedy   
1               6       4  Toy Story (1995)  Animation|Children's|Comedy   
1               8       4  Toy Story (1995)  Animation|Children's|Comedy   
1               9       5  Toy Story (1995)  Animation|Children's|Comedy   
1              10       5  Toy Story (1995)  Animation|Children's|Comedy   

          year short_title  Action  Adventure  Comedy  Drama  Romance  
movie_id                                                               
1         1995   Toy Story       0          0       1      0        0  
1         1995   Toy Story       0          0       1      0        0  
1         1995   Toy Story       0          0       1      0        0  
1         1995   Toy Story       0          0       1      0        0  
1         1995   Toy Story       0 

In [7]:
full_movies_ratings.reset_index(inplace=True)
full_movies_ratings.shape

(1000209, 12)

#### Merging users datafram with full_movies_ratings on user_id as common column
> ***full_users_ratings_movies*** dataframe

In [8]:
full_users_ratings_movies = pd.merge(users, full_movies_ratings, on=['user_id'], how='inner').drop(["occupation", "zip","gender"],axis=1)
full_users_ratings_movies.head()

   user_id  age  movie_id  rating                                      title  \
0        1    1         1       5                           Toy Story (1995)   
1        1    1        48       5                          Pocahontas (1995)   
2        1    1       150       5                           Apollo 13 (1995)   
3        1    1       260       4  Star Wars: Episode IV - A New Hope (1977)   
4        1    1       527       5                    Schindler's List (1993)   

                                 genres  year  \
0           Animation|Children's|Comedy  1995   
1  Animation|Children's|Musical|Romance  1995   
2                                 Drama  1995   
3       Action|Adventure|Fantasy|Sci-Fi  1977   
4                             Drama|War  1993   

                          short_title  Action  Adventure  Comedy  Drama  \
0                           Toy Story       0          0       1      0   
1                          Pocahontas       0          0       0      0   

In [9]:
full_users_ratings_movies.shape

(1000209, 13)